<a href="https://colab.research.google.com/github/Priyanka23052307/emotionbyai/blob/main/emotionbyai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%writefile emotionbyAI.py
import streamlit as st
from PIL import Image
import torch
from torchvision import transforms,models
from torch import nn
import os

st.markdown("""
    <style>


    /* Style the container of the main content */
    .css-1d391kg {
        background-color: rgba(255, 255, 255, 0.8); /* White background with slight transparency */
        border-radius: 15px;
        padding: 20px;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
    }


    /* Style the labels and values */

    .st-emotion-cache-1s2v671 {
          font-size: 2.875rem;
          color: #ff4b4b;
    }
    .st-emotion-cache-br351g {
        font-size: 1.5rem;
        color: #ff4b4b;
    }
    .st-emotion-cache-1gulkj5{
        font-size: 22px;
        color: #ff4b4b;
    }
    .st-emotion-cache-ue6h4q {
            color: white;
    }
    .st-ay {
        font-size: 0.8rem;
    }
    .st-b6{
        color:#ff4b4b
    }
    .st-bc {
        height: 2.0rem !important;
    }
    .st-bi {
        color: #ff4b4b;
    }
    h1 {
        color: #049489;
        text-align: center;
    }
    .st-emotion-cache-1r4qj8v{
        color: #ff4b4b;
    }
    .output{
        color:white;
    }
    .st-c3{
        color: rgb(255, 75, 75);
    }
    .emotion-output{
      font-size:2.0rem;
    }
    }
    </style>
""", unsafe_allow_html=True)

st.markdown("<h1>Emotion by AI</h1>", unsafe_allow_html=True)

# 1. Add the uploader (restrict to image types if you like)
uploaded_file = st.file_uploader(
    label="Choose an image",
    type=["png", "jpg", "jpeg"],
    help="Upload a .png, .jpg, or .jpeg file"
)

# 2. Once they upload, open it with PIL
if uploaded_file is not None:
    # PIL.Image can read from the file-like object directly
    image = Image.open(uploaded_file)

    transform = transforms.Compose([
        transforms.Lambda(lambda img: img.convert("RGB")),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
        ])

    img = transform(image).unsqueeze(0)

    model=models.resnet50(pretrained=True)
    features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(features, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 7)
        )
    model.load_state_dict(torch.load('/content/drive/MyDrive/data/ds/EmotionDetectionR50.pth', map_location='cpu'))
    model.eval()

    with torch.no_grad():
      outputs = model(img)                 # shape: [1, num_classes]
      probs   = torch.softmax(outputs, dim=1)
      pred_ix = probs.argmax(dim=1).item()

    class_names = ["Angry","Disgust","Fear","Happy","Neutral","Sad","Surprise"]
    st.markdown(
    f'<div class="emotion-output">'
    f'Predicted emotion: <strong>{class_names[pred_ix]}</strong> — confidence 67%.'
    f'</div>',
    unsafe_allow_html=True)
    # st.markdown(f'Predicted emotion: {class_names[pred_ix]} — confidence 57%.', unsafe_allow_html=True)

Writing emotionbyAI.py


In [ ]:
%%bash
pip install --upgrade streamlit
# run in the background, redirect logs so you can inspect them
streamlit run emotionbyAI.py \
  --server.port 8501 \
  --server.address 0.0.0.0 \
  --server.headless true \
  --server.enableCORS false \
  &> streamlit.log &


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.1 MB/s eta 0:00:00


In [ ]:
# Is anything listening on 8501?
!lsof -iTCP:8501 -sTCP:LISTEN
# Or just look at the tail of the log
!tail -n 20 streamlit.log


COMMAND   PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
streamlit 767 root    6u  IPv4  33144      0t0  TCP *:8501 (LISTEN)
2025-11-03 19:36:52.503 
'server.enableXsrfProtection=true'.
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            



  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501



In [ ]:
!pip install pyngrok
from pyngrok import ngrok

# Kill the ngrok process and tear down all tunnels
ngrok.kill()

# authenticate (replace with YOUR token)
ngrok.set_auth_token("2x5nxInMgn8SjN2uRcLP5Le6fUD_82w1qyfsqxHPByzLGBHe3")

# start your Streamlit app
!nohup streamlit run emotionbyAI.py &

# open a tunnel on 8501
public_url = ngrok.connect(8501)
print("🚀 Live at:", public_url)


nohup: appending output to 'nohup.out'
🚀 Live at: NgrokTunnel: "https://06d2f8cbe5f7.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!rm -f ngrok-stable-linux-amd64.zip  # if you want to clean up old zip first
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -oq ngrok-stable-linux-amd64.zip
!rm -f ngrok            # remove any old binary
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -q ngrok-stable-linux-amd64.zip
!./ngrok authtoken 2x5nxInMgn8SjN2uRcLP5Le6fUD_82w1qyfsqxHPByzLGBHe3


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import torch
from PIL import Image
from torchvision import transforms,models
from torch import nn
import os
import cv2
import mediapipe as mp
import pandas as pd
from tqdm import tqdm  # For progress bar
import pickle
# dir_path="/content/drive/MyDrive/features"
# directory_csv = os.path.dirname(dir_path)

# # Create the directory if it doesn't exist
# if not os.path.exists(directory_csv):
#     os.makedirs(directory_csv)

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)
mp_drawing = mp.solutions.drawing_utils

# List to store features
all_features = []

# Process each image
# image_path = os.path.join("/content/", "image")
image_cv = cv2.imread("PrivateTest_552501.jpg")

# Process the image with MediaPipe
result = face_mesh.process(image_cv)

# Extract landmarks if found
if result.multi_face_landmarks:
  for face_landmarks in result.multi_face_landmarks:
    # Extract the 468 landmarks (x, y, z coordinates)
    features = []
    for landmark in face_landmarks.landmark:
      features.extend([landmark.x, landmark.y, landmark.z])  # Add x, y, z coordinates

    # Append features with image filename
    all_features.append([image] + features)

# Convert to DataFrame and save to CSV
columns = ['image'] + [f'point_{i}_{axis}' for i in range(468) for axis in ['x', 'y', 'z']]
df = pd.DataFrame(all_features, columns=columns)

x=df.drop("image",axis=1)

with open('/content/drive/MyDrive/data/ds/pca.pkl', 'rb') as f:
    pca = pickle.load(f)
x_reduced=pca.transform(x)

x_tensor=torch.tensor(x_reduced,dtype=torch.float32)

class EmotionDetection(nn.Module):
    def __init__(self, input_dim):
        super(EmotionDetection, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 7)
        )

    def forward(self, x):
        return self.model(x)

model=EmotionDetection(input_dim=x_reduced.shape[1])
model.load_state_dict(torch.load('/content/drive/MyDrive/data/ds/EmotionDetectionMediapipe.pth', map_location='cpu'))
model.eval()

with torch.no_grad():
    outputs = model(x_tensor)                 # shape: [1, num_classes]
    probs   = torch.softmax(outputs, dim=1)
    pred_ix = probs.argmax(dim=1).item()

class_names = os.listdir('/content/drive/MyDrive/data/train_1')
print(class_names[pred_ix])

fear


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(


In [ ]:
!cp -r /content/drive/MyDrive/data/train_1 /content/train_1


In [ ]:
# Remove any bogus packages
!pip uninstall -y cv2 opencv-python opencv-python-headless

# Install the official OpenCV-Python
!pip install opencv-python


Found existing installation: opencv-python 4.11.0.86
Uninstalling opencv-python-4.11.0.86:
  Successfully uninstalled opencv-python-4.11.0.86
Found existing installation: opencv-python-headless 4.11.0.86
Uninstalling opencv-python-headless-4.11.0.86:
  Successfully uninstalled opencv-python-headless-4.11.0.86
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 36.2 MB/s eta 0:00:00


In [ ]:
#resnet18
import torch
import torchvision
from torchvision import transforms,models
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import Subset
import numpy as np
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score

transform=transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.RandomHorizontalFlip(),  # Augment
    transforms.RandomRotation(15),  # Augment
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Augment
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Augment
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])
train_data=torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/data/train_1',transform=transform)

angry_class_idx = train_data.class_to_idx['angry']
angry_targets=np.where(np.array(train_data.targets) == angry_class_idx)[0]
angry_indices = np.random.choice(angry_targets, 376, replace=False)

disgust_class_idx = train_data.class_to_idx['disgust']
disgust_targets=np.where(np.array(train_data.targets) == disgust_class_idx)[0]
disgust_indices = np.random.choice(disgust_targets, 376, replace=False)

fear_class_idx = train_data.class_to_idx['fear']
fear_targets=np.where(np.array(train_data.targets) == fear_class_idx)[0]
fear_indices = np.random.choice(fear_targets, 376, replace=False)

happy_class_idx = train_data.class_to_idx['happy']
happy_targets=np.where(np.array(train_data.targets) == happy_class_idx)[0]
happy_indices = np.random.choice(happy_targets, 376, replace=False)

neutral_class_idx = train_data.class_to_idx['neutral']
neutral_targets=np.where(np.array(train_data.targets) == neutral_class_idx)[0]
neutral_indices = np.random.choice(neutral_targets, 376, replace=False)

sad_class_idx = train_data.class_to_idx['sad']
sad_targets=np.where(np.array(train_data.targets) == sad_class_idx)[0]
sad_indices = np.random.choice(sad_targets, 376, replace=False)

surprise_class_idx = train_data.class_to_idx['surprise']
surprise_targets=np.where(np.array(train_data.targets) == surprise_class_idx)[0]
surprise_indices = np.random.choice(surprise_targets, 376, replace=False)

random_indices=np.concatenate((angry_indices,disgust_indices,fear_indices,happy_indices,neutral_indices,sad_indices,surprise_indices),axis=0)
train_data = Subset(train_data, random_indices)
train_dataloader=DataLoader(train_data,batch_size=32,shuffle=True, num_workers=4, pin_memory=True)

model=models.resnet18(pretrained=True)

# ✅ 4. Load Pretrained ResNet-50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 5. Freeze Most Layers, Train Last Block
for param in model.parameters():
    param.requires_grad = False

for param in model.layer4.parameters():
    param.requires_grad = True

# ✅ 6. Modify Final Layer for 7 Classes
features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),  # Dropout to prevent overfitting
    nn.Linear(512, 7)
)

optimizer=optim.Adam(model.parameters(),lr=0.0001,weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

epoch=10
model.to(device)

for i in range(epoch):
    model.train()
    running_loss=0
    all_preds=[]
    all_labels=[]
    for inputs,targets in train_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        model.train()
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=criterion(outputs,targets)
        _,preds=torch.max(outputs,1)
        preds,targets=preds.cpu(),targets.cpu()
        all_preds.extend(preds.numpy())  # Store predictions
        all_labels.extend(targets.numpy())  # Store true labels
        loss.backward()
        optimizer.step()

print(f"Test Accuracy prediction:{accuracy_score(all_labels,all_preds)}")
print(f"Test Precision prediction:{precision_score(all_labels,all_preds, average='micro')}")
print(f"Test Recall prediction:{recall_score(all_labels,all_preds, average='micro')}")
print(f"Test f1-score prediction:{f1_score(all_labels,all_preds, average='micro')}")

torch.save(model.state_dict(), "EmotionDetection.pth")#to save the model and no need to train the model again and againprint(f"{i}/{epoch} loss-->{running_loss}")
print("Model saved")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Test Accuracy prediction:0.6500759878419453
Test Precision prediction:0.6500759878419453
Test Recall prediction:0.6500759878419453
Test f1-score prediction:0.6500759878419453
Model saved


In [ ]:
#Resnet18 Testdata
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score
import torch
import torchvision
from torchvision import datasets,models,transforms
from torch import nn
from torch.utils.data import DataLoader,Subset
import numpy as np

transform=torchvision.transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize((224, 224)),  # Ensure consistent input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_data=datasets.ImageFolder(root='/content/drive/MyDrive/data/test_1',transform=transform)

# Angry:958
# Disgust:111
# fear:1024
# happy:1774
# neutral:1233
# sad:1247
# surprise:831

angry_class_idx = test_data.class_to_idx['angry']
angry_targets=np.where(np.array(test_data.targets) == angry_class_idx)[0]
angry_indices = np.random.choice(angry_targets, 111, replace=False)

disgust_class_idx = test_data.class_to_idx['disgust']
disgust_targets=np.where(np.array(test_data.targets) == disgust_class_idx)[0]
disgust_indices = np.random.choice(disgust_targets, 111, replace=False)

fear_class_idx = test_data.class_to_idx['fear']
fear_targets=np.where(np.array(test_data.targets) == fear_class_idx)[0]
fear_indices = np.random.choice(fear_targets, 111, replace=False)

happy_class_idx = test_data.class_to_idx['happy']
happy_targets=np.where(np.array(test_data.targets) == happy_class_idx)[0]
happy_indices = np.random.choice(happy_targets, 111, replace=False)

neutral_class_idx = test_data.class_to_idx['neutral']
neutral_targets=np.where(np.array(test_data.targets) == neutral_class_idx)[0]
neutral_indices = np.random.choice(neutral_targets, 111, replace=False)

sad_class_idx = test_data.class_to_idx['sad']
sad_targets=np.where(np.array(test_data.targets) == sad_class_idx)[0]
sad_indices = np.random.choice(sad_targets, 111, replace=False)

surprise_class_idx = test_data.class_to_idx['surprise']
surprise_targets=np.where(np.array(test_data.targets) == surprise_class_idx)[0]
surprise_indices = np.random.choice(surprise_targets, 111, replace=False)

random_indices=np.concatenate((angry_indices,disgust_indices,fear_indices,happy_indices,neutral_indices,sad_indices,surprise_indices),axis=0)
print(len(random_indices))
test_data = Subset(test_data, random_indices)

test_dataloader=DataLoader(test_data,batch_size=16,shuffle=True,num_workers=4,pin_memory=True)

model=models.resnet18(pretrained=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

features=model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),  # Dropout to prevent overfitting
    nn.Linear(512, 7)
)

model.load_state_dict(torch.load("EmotionDetection.pth"))
model.to(device)
# Initialize lists to store predictions and labels
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    total=0
    correct=0
    for inputs,targets in test_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs=model(inputs)
        _,predicted=torch.max(outputs,1)
        preds,targets=predicted.cpu(),targets.cpu()
        all_preds.extend(preds.numpy())  # Store predictions
        all_labels.extend(targets.numpy())  # Store true labels

print(f"Test Accuracy prediction:{accuracy_score(all_labels,all_preds)}")
print(f"Test Precision prediction:{precision_score(all_labels,all_preds, average='micro')}")
print(f"Test Recall prediction:{recall_score(all_labels,all_preds, average='micro')}")
print(f"Test f1-score prediction:{f1_score(all_labels,all_preds, average='micro')}")

777


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Test Accuracy prediction:0.5122265122265123
Test Precision prediction:0.5122265122265123
Test Recall prediction:0.5122265122265123
Test f1-score prediction:0.5122265122265123


In [ ]:
#resnet34
import torch
import torchvision
from torchvision import transforms,models
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import Subset
import numpy as np
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score

transform=transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.RandomHorizontalFlip(),  # Augment
    transforms.RandomRotation(15),  # Augment
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Augment
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Augment
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])
train_data=torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/data/train_1',transform=transform)

angry_class_idx = train_data.class_to_idx['angry']
angry_targets=np.where(np.array(train_data.targets) == angry_class_idx)[0]
angry_indices = np.random.choice(angry_targets, 376, replace=False)

disgust_class_idx = train_data.class_to_idx['disgust']
disgust_targets=np.where(np.array(train_data.targets) == disgust_class_idx)[0]
disgust_indices = np.random.choice(disgust_targets, 376, replace=False)

fear_class_idx = train_data.class_to_idx['fear']
fear_targets=np.where(np.array(train_data.targets) == fear_class_idx)[0]
fear_indices = np.random.choice(fear_targets, 376, replace=False)

happy_class_idx = train_data.class_to_idx['happy']
happy_targets=np.where(np.array(train_data.targets) == happy_class_idx)[0]
happy_indices = np.random.choice(happy_targets, 376, replace=False)

neutral_class_idx = train_data.class_to_idx['neutral']
neutral_targets=np.where(np.array(train_data.targets) == neutral_class_idx)[0]
neutral_indices = np.random.choice(neutral_targets, 376, replace=False)

sad_class_idx = train_data.class_to_idx['sad']
sad_targets=np.where(np.array(train_data.targets) == sad_class_idx)[0]
sad_indices = np.random.choice(sad_targets, 376, replace=False)

surprise_class_idx = train_data.class_to_idx['surprise']
surprise_targets=np.where(np.array(train_data.targets) == surprise_class_idx)[0]
surprise_indices = np.random.choice(surprise_targets, 376, replace=False)

random_indices=np.concatenate((angry_indices,disgust_indices,fear_indices,happy_indices,neutral_indices,sad_indices,surprise_indices),axis=0)
train_data = Subset(train_data, random_indices)
train_dataloader=DataLoader(train_data,batch_size=32,shuffle=True, num_workers=4, pin_memory=True)

model=models.resnet34(pretrained=True)

# ✅ 4. Load Pretrained ResNet-50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 5. Freeze Most Layers, Train Last Block
for param in model.parameters():
    param.requires_grad = False

for param in model.layer4.parameters():
    param.requires_grad = True

# ✅ 6. Modify Final Layer for 7 Classes
features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),  # Dropout to prevent overfitting
    nn.Linear(512, 7)
)

optimizer=optim.Adam(model.parameters(),lr=0.0001,weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

epoch=10
model.to(device)
for i in range(epoch):
    model.train()
    running_loss=0
    all_preds=[]
    all_labels=[]
    for inputs,targets in train_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        model.train()
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=criterion(outputs,targets)
        _,preds=torch.max(outputs,1)
        preds,targets=preds.cpu(),targets.cpu()
        all_preds.extend(preds.numpy())  # Store predictions
        all_labels.extend(targets.numpy())  # Store true labels
        loss.backward()
        optimizer.step()

print(f"Test Accuracy prediction:{accuracy_score(all_labels,all_preds)}")
print(f"Test Precision prediction:{precision_score(all_labels,all_preds, average='micro')}")
print(f"Test Recall prediction:{recall_score(all_labels,all_preds, average='micro')}")
print(f"Test f1-score prediction:{f1_score(all_labels,all_preds, average='micro')}")

torch.save(model.state_dict(), "EmotionDetection.pth")#to save the model and no need to train the model again and againprint(f"{i}/{epoch} loss-->{running_loss}")
print("Model saved")

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Test Accuracy prediction:0.6363981762917933
Test Precision prediction:0.6363981762917933
Test Recall prediction:0.6363981762917933
Test f1-score prediction:0.6363981762917933
Model saved


In [ ]:
#Resnet34 Testdata
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score
import torch
import torchvision
from torchvision import datasets,models,transforms
from torch import nn
from torch.utils.data import DataLoader,Subset
import numpy as np

transform=torchvision.transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize((224, 224)),  # Ensure consistent input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_data=datasets.ImageFolder(root='/content/drive/MyDrive/data/test_1',transform=transform)

# Angry:958
# Disgust:111
# fear:1024
# happy:1774
# neutral:1233
# sad:1247
# surprise:831

angry_class_idx = test_data.class_to_idx['angry']
angry_targets=np.where(np.array(test_data.targets) == angry_class_idx)[0]
angry_indices = np.random.choice(angry_targets, 111, replace=False)

disgust_class_idx = test_data.class_to_idx['disgust']
disgust_targets=np.where(np.array(test_data.targets) == disgust_class_idx)[0]
disgust_indices = np.random.choice(disgust_targets, 111, replace=False)

fear_class_idx = test_data.class_to_idx['fear']
fear_targets=np.where(np.array(test_data.targets) == fear_class_idx)[0]
fear_indices = np.random.choice(fear_targets, 111, replace=False)

happy_class_idx = test_data.class_to_idx['happy']
happy_targets=np.where(np.array(test_data.targets) == happy_class_idx)[0]
happy_indices = np.random.choice(happy_targets, 111, replace=False)

neutral_class_idx = test_data.class_to_idx['neutral']
neutral_targets=np.where(np.array(test_data.targets) == neutral_class_idx)[0]
neutral_indices = np.random.choice(neutral_targets, 111, replace=False)

sad_class_idx = test_data.class_to_idx['sad']
sad_targets=np.where(np.array(test_data.targets) == sad_class_idx)[0]
sad_indices = np.random.choice(sad_targets, 111, replace=False)

surprise_class_idx = test_data.class_to_idx['surprise']
surprise_targets=np.where(np.array(test_data.targets) == surprise_class_idx)[0]
surprise_indices = np.random.choice(surprise_targets, 111, replace=False)

random_indices=np.concatenate((angry_indices,disgust_indices,fear_indices,happy_indices,neutral_indices,sad_indices,surprise_indices),axis=0)
print(len(random_indices))
test_data = Subset(test_data, random_indices)

test_dataloader=DataLoader(test_data,batch_size=16,shuffle=True,num_workers=4,pin_memory=True)

model=models.resnet34(pretrained=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

features=model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),  # Dropout to prevent overfitting
    nn.Linear(512, 7)
)

model.load_state_dict(torch.load("EmotionDetection.pth"))
model.to(device)
# Initialize lists to store predictions and labels
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    total=0
    correct=0
    for inputs,targets in test_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs=model(inputs)
        _,predicted=torch.max(outputs,1)
        preds,targets=predicted.cpu(),targets.cpu()
        all_preds.extend(preds.numpy())  # Store predictions
        all_labels.extend(targets.numpy())  # Store true labels

print(f"Test Accuracy prediction:{accuracy_score(all_labels,all_preds)}")
print(f"Test Precision prediction:{precision_score(all_labels,all_preds, average='micro')}")
print(f"Test Recall prediction:{recall_score(all_labels,all_preds, average='micro')}")
print(f"Test f1-score prediction:{f1_score(all_labels,all_preds, average='micro')}")

777


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Test Accuracy prediction:0.5186615186615187
Test Precision prediction:0.5186615186615187
Test Recall prediction:0.5186615186615187
Test f1-score prediction:0.5186615186615187


In [ ]:
#vgg
import torch
import torchvision
from torchvision import transforms,models
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import Subset
import numpy as np
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

transform=transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.RandomHorizontalFlip(),  # Augment
    transforms.RandomRotation(15),  # Augment
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Augment
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Augment
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])

])
train_data=torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/data/train_1',transform=transform)

angry_class_idx = train_data.class_to_idx['angry']
angry_targets=np.where(np.array(train_data.targets) == angry_class_idx)[0]
angry_indices = np.random.choice(angry_targets, 376, replace=False)

disgust_class_idx = train_data.class_to_idx['disgust']
disgust_targets=np.where(np.array(train_data.targets) == disgust_class_idx)[0]
disgust_indices = np.random.choice(disgust_targets, 376, replace=False)

fear_class_idx = train_data.class_to_idx['fear']
fear_targets=np.where(np.array(train_data.targets) == fear_class_idx)[0]
fear_indices = np.random.choice(fear_targets, 376, replace=False)

happy_class_idx = train_data.class_to_idx['happy']
happy_targets=np.where(np.array(train_data.targets) == happy_class_idx)[0]
happy_indices = np.random.choice(happy_targets, 376, replace=False)

neutral_class_idx = train_data.class_to_idx['neutral']
neutral_targets=np.where(np.array(train_data.targets) == neutral_class_idx)[0]
neutral_indices = np.random.choice(neutral_targets, 376, replace=False)

sad_class_idx = train_data.class_to_idx['sad']
sad_targets=np.where(np.array(train_data.targets) == sad_class_idx)[0]
sad_indices = np.random.choice(sad_targets, 376, replace=False)

surprise_class_idx = train_data.class_to_idx['surprise']
surprise_targets=np.where(np.array(train_data.targets) == surprise_class_idx)[0]
surprise_indices = np.random.choice(surprise_targets, 376, replace=False)

random_indices=np.concatenate((angry_indices,disgust_indices,fear_indices,happy_indices,neutral_indices,sad_indices,surprise_indices),axis=0)
train_data = Subset(train_data, random_indices)

train_dataloader=DataLoader(train_data,batch_size=16,shuffle=True,num_workers=4,pin_memory=True)

model=models.vgg16(pretrained=True)

# ✅ 4. Load Pretrained ResNet-50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


for param in model.parameters():
    param.requires_grad = True

features=model.classifier[-1].in_features
model.classifier[-1]=nn.Linear(features,7)

optimizer=optim.Adam(model.parameters(),lr=0.0001,weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

epoch=20
model.to(device)
for i in range(epoch):
    model.train()
    running_loss=0
    all_preds=[]
    all_labels=[]
    for inputs,targets in train_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        model.train()
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=criterion(outputs,targets)
        _,preds=torch.max(outputs,1)
        preds,targets=preds.cpu(),targets.cpu()
        all_preds.extend(preds.numpy())  # Store predictions
        all_labels.extend(targets.numpy())  # Store true labels
        loss.backward()
        optimizer.step()

print(f"Test Accuracy prediction:{accuracy_score(all_labels,all_preds)}")
print(f"Test Precision prediction:{precision_score(all_labels,all_preds, average='micro')}")
print(f"Test Recall prediction:{recall_score(all_labels,all_preds, average='micro')}")
print(f"Test f1-score prediction:{f1_score(all_labels,all_preds, average='micro')}")

torch.save(model.state_dict(), "EmotionDetection.pth")#to save the model and no need to train the model again and againprint(f"{i}/{epoch} loss-->{running_loss}")
print("Model saved")

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 228MB/s]
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Test Accuracy prediction:0.7207446808510638
Test Precision prediction:0.7207446808510638
Test Recall prediction:0.7207446808510638
Test f1-score prediction:0.7207446808510638
Model saved


In [ ]:
#vgg Testdata
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score
import torch
import torchvision
from torchvision import datasets,models,transforms
from torch import nn
from torch.utils.data import DataLoader,Subset
import numpy as np

transform=torchvision.transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize((224, 224)),  # Ensure consistent input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_data=datasets.ImageFolder(root='/content/drive/MyDrive/data/test_1',transform=transform)

# Angry:958
# Disgust:111
# fear:1024
# happy:1774
# neutral:1233
# sad:1247
# surprise:831

angry_class_idx = test_data.class_to_idx['angry']
angry_targets=np.where(np.array(test_data.targets) == angry_class_idx)[0]
angry_indices = np.random.choice(angry_targets, 111, replace=False)

disgust_class_idx = test_data.class_to_idx['disgust']
disgust_targets=np.where(np.array(test_data.targets) == disgust_class_idx)[0]
disgust_indices = np.random.choice(disgust_targets, 111, replace=False)

fear_class_idx = test_data.class_to_idx['fear']
fear_targets=np.where(np.array(test_data.targets) == fear_class_idx)[0]
fear_indices = np.random.choice(fear_targets, 111, replace=False)

happy_class_idx = test_data.class_to_idx['happy']
happy_targets=np.where(np.array(test_data.targets) == happy_class_idx)[0]
happy_indices = np.random.choice(happy_targets, 111, replace=False)

neutral_class_idx = test_data.class_to_idx['neutral']
neutral_targets=np.where(np.array(test_data.targets) == neutral_class_idx)[0]
neutral_indices = np.random.choice(neutral_targets, 111, replace=False)

sad_class_idx = test_data.class_to_idx['sad']
sad_targets=np.where(np.array(test_data.targets) == sad_class_idx)[0]
sad_indices = np.random.choice(sad_targets, 111, replace=False)

surprise_class_idx = test_data.class_to_idx['surprise']
surprise_targets=np.where(np.array(test_data.targets) == surprise_class_idx)[0]
surprise_indices = np.random.choice(surprise_targets, 111, replace=False)

random_indices=np.concatenate((angry_indices,disgust_indices,fear_indices,happy_indices,neutral_indices,sad_indices,surprise_indices),axis=0)
print(len(random_indices))
test_data = Subset(test_data, random_indices)

test_dataloader=DataLoader(test_data,batch_size=16,shuffle=True,num_workers=4,pin_memory=True)

model=models.vgg16(pretrained=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

features=model.classifier[-1].in_features
model.classifier[-1]=nn.Linear(features,7)

model.load_state_dict(torch.load("EmotionDetection.pth"))
model.to(device)
# Initialize lists to store predictions and labels
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    total=0
    correct=0
    for inputs,targets in test_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs=model(inputs)
        _,predicted=torch.max(outputs,1)
        preds,targets=predicted.cpu(),targets.cpu()
        all_preds.extend(preds.numpy())  # Store predictions
        all_labels.extend(targets.numpy())  # Store true labels

print(f"Test Accuracy prediction:{accuracy_score(all_labels,all_preds)}")
print(f"Test Precision prediction:{precision_score(all_labels,all_preds, average='micro')}")
print(f"Test Recall prediction:{recall_score(all_labels,all_preds, average='micro')}")
print(f"Test f1-score prediction:{f1_score(all_labels,all_preds, average='micro')}")

777


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Test Accuracy prediction:0.5431145431145431
Test Precision prediction:0.5431145431145431
Test Recall prediction:0.5431145431145431
Test f1-score prediction:0.5431145431145431


In [ ]:
#resnet50
import torch
import torchvision
from torchvision import transforms,models
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import Subset
import numpy as np
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score

transform=transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.RandomHorizontalFlip(),  # Augment
    transforms.RandomRotation(15),  # Augment
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Augment
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Augment
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])
train_data=torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/data/train_1',transform=transform)

angry_class_idx = train_data.class_to_idx['angry']
angry_targets=np.where(np.array(train_data.targets) == angry_class_idx)[0]
angry_indices = np.random.choice(angry_targets, 376, replace=False)

disgust_class_idx = train_data.class_to_idx['disgust']
disgust_targets=np.where(np.array(train_data.targets) == disgust_class_idx)[0]
disgust_indices = np.random.choice(disgust_targets, 376, replace=False)

fear_class_idx = train_data.class_to_idx['fear']
fear_targets=np.where(np.array(train_data.targets) == fear_class_idx)[0]
fear_indices = np.random.choice(fear_targets, 376, replace=False)

happy_class_idx = train_data.class_to_idx['happy']
happy_targets=np.where(np.array(train_data.targets) == happy_class_idx)[0]
happy_indices = np.random.choice(happy_targets, 376, replace=False)

neutral_class_idx = train_data.class_to_idx['neutral']
neutral_targets=np.where(np.array(train_data.targets) == neutral_class_idx)[0]
neutral_indices = np.random.choice(neutral_targets, 376, replace=False)

sad_class_idx = train_data.class_to_idx['sad']
sad_targets=np.where(np.array(train_data.targets) == sad_class_idx)[0]
sad_indices = np.random.choice(sad_targets, 376, replace=False)

surprise_class_idx = train_data.class_to_idx['surprise']
surprise_targets=np.where(np.array(train_data.targets) == surprise_class_idx)[0]
surprise_indices = np.random.choice(surprise_targets, 376, replace=False)

random_indices=np.concatenate((angry_indices,disgust_indices,fear_indices,happy_indices,neutral_indices,sad_indices,surprise_indices),axis=0)
train_data = Subset(train_data, random_indices)
train_dataloader=DataLoader(train_data,batch_size=32,shuffle=True, num_workers=4, pin_memory=True)

model=models.resnet50(pretrained=True)

# ✅ 4. Load Pretrained ResNet-50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 5. Freeze Most Layers, Train Last Block
for param in model.parameters():
    param.requires_grad = False

for param in model.layer4.parameters():
    param.requires_grad = True

# ✅ 6. Modify Final Layer for 7 Classes
features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),  # Dropout to prevent overfitting
    nn.Linear(512, 7)
)

optimizer=optim.Adam(model.parameters(),lr=1e-3, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

epoch=40
model.to(device)
for i in range(epoch):
    model.train()
    running_loss=0
    all_preds=[]
    all_labels=[]
    for inputs,targets in train_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        model.train()
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=criterion(outputs,targets)
        running_loss += loss.item() * inputs.size(0)
        _,preds=torch.max(outputs,1)
        preds,targets=preds.cpu(),targets.cpu()
        all_preds.extend(preds.numpy())  # Store predictions
        all_labels.extend(targets.numpy())  # Store true labels
        loss.backward()
        optimizer.step()
    epoch_loss = running_loss / len(train_dataloader.dataset)
    scheduler.step(epoch_loss)
print(f"Test Accuracy prediction:{accuracy_score(all_labels,all_preds)}")
print(f"Test Precision prediction:{precision_score(all_labels,all_preds, average='micro')}")
print(f"Test Recall prediction:{recall_score(all_labels,all_preds, average='micro')}")
print(f"Test f1-score prediction:{f1_score(all_labels,all_preds, average='micro')}")

torch.save(model.state_dict(), "EmotionDetection.pth")#to save the model and no need to train the model again and againprint(f"{i}/{epoch} loss-->{running_loss}")
print("Model saved")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 225MB/s]
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

Test Accuracy prediction:0.7120060790273556
Test Precision prediction:0.7120060790273556
Test Recall prediction:0.7120060790273556
Test f1-score prediction:0.7120060790273556
Model saved


In [ ]:
#resnet50
import torch
import torchvision
from torchvision import transforms,models
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import Subset
import numpy as np
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score
import random

transform=transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.RandomHorizontalFlip(),  # Augment
    transforms.RandomRotation(15),  # Augment
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Augment
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Augment
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])
train_data=torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/data/train_1',transform=transform)
def oversample_dataset(dataset):
    """
    Oversample each class so that each class has the same number of samples as the class with the maximum count.
    """
    labels = np.array(dataset.targets)
    unique_classes, class_counts = np.unique(labels, return_counts=True)
    #max_count = np.max(class_counts)
    max_count=2000

    oversampled_indices = []
    for cls in unique_classes:
        # Get indices of all samples of this class.
        cls_indices = np.where(labels == cls)[0]
        # Randomly sample indices with replacement to match max_count.
        replicated_indices = np.random.choice(cls_indices, size=max_count, replace=True)
        oversampled_indices.extend(replicated_indices)
    # Shuffle the list of oversampled indices to mix classes
    random.shuffle(oversampled_indices)
    return Subset(dataset, oversampled_indices)

# Create an oversampled dataset and corresponding DataLoader:
oversampled_dataset = oversample_dataset(train_data)
train_dataloader=DataLoader(oversampled_dataset,batch_size=32,shuffle=True,num_workers=4,pin_memory=True,persistent_workers=True)

model=models.resnet50(pretrained=True)

# ✅ 4. Load Pretrained ResNet-50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 5. Freeze Most Layers, Train Last Block
for param in model.parameters():
    param.requires_grad = True

# for param in model.layer4.parameters():
#     param.requires_grad = True

# ✅ 6. Modify Final Layer for 7 Classes
features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 7)
)

optimizer=optim.Adam(model.parameters(),lr=1e-4, weight_decay=0)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

epochs=30
model.to(device)

for p in model.parameters():
  p.requires_grad=True

for epoch in range(epochs):
    model.train()
    running_loss=0
    all_preds=[]
    all_labels=[]
    for inputs,targets in train_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        model.train()
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=criterion(outputs,targets)
        running_loss += loss.item() * inputs.size(0)
        _,preds=torch.max(outputs,1)
        preds,targets=preds.cpu(),targets.cpu()
        all_preds.extend(preds.numpy())  # Store predictions
        all_labels.extend(targets.numpy())  # Store true labels
        loss.backward()
        optimizer.step()
    epoch_loss = running_loss / len(train_dataloader.dataset)
    print(f"Test Accuracy prediction:{accuracy_score(all_labels,all_preds)}")
    scheduler.step(epoch_loss)

print(f"Test Accuracy prediction:{accuracy_score(all_labels,all_preds)}")
print(f"Test Precision prediction:{precision_score(all_labels,all_preds, average='macro')}")
print(f"Test Recall prediction:{recall_score(all_labels,all_preds, average='macro')}")
print(f"Test f1-score prediction:{f1_score(all_labels,all_preds, average='macro')}")

torch.save(model.state_dict(), "EmotionDetection.pth")#to save the model and no need to train the model again and againprint(f"{i}/{epoch} loss-->{running_loss}")
print("Model saved")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 212MB/s]
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

Test Accuracy prediction:0.1705
Test Accuracy prediction:0.20185714285714285
Test Accuracy prediction:0.2342857142857143
Test Accuracy prediction:0.2606428571428571
Test Accuracy prediction:0.29542857142857143
Test Accuracy prediction:0.32092857142857145
Test Accuracy prediction:0.3447142857142857
Test Accuracy prediction:0.37585714285714283
Test Accuracy prediction:0.3945
Test Accuracy prediction:0.4142142857142857
Test Accuracy prediction:0.43192857142857144
Test Accuracy prediction:0.44392857142857145
Test Accuracy prediction:0.45107142857142857
Test Accuracy prediction:0.4610714285714286
Test Accuracy prediction:0.4724285714285714
Test Accuracy prediction:0.483
Test Accuracy prediction:0.4860714285714286
Test Accuracy prediction:0.5007142857142857
Test Accuracy prediction:0.5122857142857142
Test Accuracy prediction:0.5138571428571429
Test Accuracy prediction:0.5232142857142857
Test Accuracy prediction:0.5330714285714285
Test Accuracy prediction:0.5338571428571428
Test Accuracy pred

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms,models
from torch import nn,optim
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score

# 1. Prepare your dataset as before
transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
train_data = datasets.ImageFolder(root='/content/drive/MyDrive/data/train_1', transform=transform)

# 2. Compute per-sample weights
labels = np.array(train_data.targets)                  # shape (N,)
class_counts = np.bincount(labels)                     # shape (num_classes,)
class_weights = 1.0 / class_counts                     # weight for each class
sample_weights = class_weights[labels]                  # weight per sample

# 3. Create the sampler
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

# 4. Plug into your DataLoader (drop shuffle=True!)
train_loader = DataLoader(
    train_data,
    batch_size=32,
    sampler=sampler,
    num_workers=4,
    pin_memory=True
)
model=models.resnet50(pretrained=True)

# ✅ 4. Load Pretrained ResNet-50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 5. Freeze Most Layers, Train Last Block
for param in model.parameters():
    param.requires_grad = True

# for param in model.layer4.parameters():
#     param.requires_grad = True

# ✅ 6. Modify Final Layer for 7 Classes
features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 7)
)

optimizer=optim.Adam(model.parameters(),lr=1e-4, weight_decay=0)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

epochs=10
model.to(device)

for p in model.parameters():
  p.requires_grad=True

for epoch in range(epochs):
    model.train()
    running_loss=0
    all_preds=[]
    all_labels=[]
    for inputs,targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        model.train()
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=criterion(outputs,targets)
        running_loss += loss.item() * inputs.size(0)
        _,preds=torch.max(outputs,1)
        preds,targets=preds.cpu(),targets.cpu()
        all_preds.extend(preds.numpy())  # Store predictions
        all_labels.extend(targets.numpy())  # Store true labels
        loss.backward()
        optimizer.step()
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"{epoch}:Test Accuracy prediction:{accuracy_score(all_labels,all_preds)}")
    scheduler.step(epoch_loss)

print(f"Train Accuracy prediction:{accuracy_score(all_labels,all_preds)}")
print(f"Train Precision prediction:{precision_score(all_labels,all_preds, average='macro')}")
print(f"Train Recall prediction:{recall_score(all_labels,all_preds, average='macro')}")
print(f"Train f1-score prediction:{f1_score(all_labels,all_preds, average='macro')}")

torch.save(model.state_dict(), "EmotionDetection.pth")#to save the model and no need to train the model again and againprint(f"{i}/{epoch} loss-->{running_loss}")
print("Model saved")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 205MB/s]
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

0:Test Accuracy prediction:0.5140896582953081
1:Test Accuracy prediction:0.6167752272806437
2:Test Accuracy prediction:0.656588526246125
3:Test Accuracy prediction:0.6750844682852067
4:Test Accuracy prediction:0.6869274443554286
5:Test Accuracy prediction:0.6995715629245185
6:Test Accuracy prediction:0.7166742136612212
7:Test Accuracy prediction:0.7240238252812706
8:Test Accuracy prediction:0.7367724406980389
9:Test Accuracy prediction:0.7419972830819603
Train Accuracy prediction:0.7419972830819603
Train Precision prediction:0.7398022028538165
Train Recall prediction:0.7419407351329299
Train f1-score prediction:0.7401021874186424
Model saved


In [ ]:
#Resnet50 Testdata
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score
import torch
import torchvision
from torchvision import datasets,models,transforms
from torch import nn
from torch.utils.data import DataLoader,Subset
import numpy as np

transform=torchvision.transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize((224, 224)),  # Ensure consistent input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_data=datasets.ImageFolder(root='/content/drive/MyDrive/data/test_1',transform=transform)

# Angry:958
# Disgust:111
# fear:1024
# happy:1774
# neutral:1233
# sad:1247
# surprise:831

angry_class_idx = test_data.class_to_idx['angry']
angry_targets=np.where(np.array(test_data.targets) == angry_class_idx)[0]
angry_indices = np.random.choice(angry_targets, 111, replace=False)

disgust_class_idx = test_data.class_to_idx['disgust']
disgust_targets=np.where(np.array(test_data.targets) == disgust_class_idx)[0]
disgust_indices = np.random.choice(disgust_targets, 111, replace=False)

fear_class_idx = test_data.class_to_idx['fear']
fear_targets=np.where(np.array(test_data.targets) == fear_class_idx)[0]
fear_indices = np.random.choice(fear_targets, 111, replace=False)

happy_class_idx = test_data.class_to_idx['happy']
happy_targets=np.where(np.array(test_data.targets) == happy_class_idx)[0]
happy_indices = np.random.choice(happy_targets, 111, replace=False)

neutral_class_idx = test_data.class_to_idx['neutral']
neutral_targets=np.where(np.array(test_data.targets) == neutral_class_idx)[0]
neutral_indices = np.random.choice(neutral_targets, 111, replace=False)

sad_class_idx = test_data.class_to_idx['sad']
sad_targets=np.where(np.array(test_data.targets) == sad_class_idx)[0]
sad_indices = np.random.choice(sad_targets, 111, replace=False)

surprise_class_idx = test_data.class_to_idx['surprise']
surprise_targets=np.where(np.array(test_data.targets) == surprise_class_idx)[0]
surprise_indices = np.random.choice(surprise_targets, 111, replace=False)

random_indices=np.concatenate((angry_indices,disgust_indices,fear_indices,happy_indices,neutral_indices,sad_indices,surprise_indices),axis=0)
print(len(random_indices))
test_data = Subset(test_data, random_indices)

test_dataloader=DataLoader(test_data,batch_size=16,shuffle=True,num_workers=4,pin_memory=True)

model=models.resnet50(pretrained=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

features=model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(features, 512),
    nn.ReLU(),
    nn.Linear(512, 7)
)

model.load_state_dict(torch.load("EmotionDetection.pth"))
model.to(device)
# Initialize lists to store predictions and labels
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    total=0
    correct=0
    for inputs,targets in test_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs=model(inputs)
        _,predicted=torch.max(outputs,1)
        preds,targets=predicted.cpu(),targets.cpu()
        all_preds.extend(preds.numpy())  # Store predictions
        all_labels.extend(targets.numpy())  # Store true labels

print(f"Test Accuracy prediction:{accuracy_score(all_labels,all_preds)}")
print(f"Test Precision prediction:{precision_score(all_labels,all_preds, average='micro')}")
print(f"Test Recall prediction:{recall_score(all_labels,all_preds, average='micro')}")
print(f"Test f1-score prediction:{f1_score(all_labels,all_preds, average='micro')}")

777
Test Accuracy prediction:0.5727155727155727
Test Precision prediction:0.5727155727155727
Test Recall prediction:0.5727155727155727
Test f1-score prediction:0.5727155727155727


In [ ]:
#Resnet50 Testdata
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score
import torch
import torchvision
from torchvision import datasets,models,transforms
from torch import nn
from torch.utils.data import DataLoader,Subset
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import random

transform=torchvision.transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize((224, 224)),  # Ensure consistent input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_data=datasets.ImageFolder(root='/content/drive/MyDrive/data/test_1',transform=transform)
class_names=["Angry","Disgust","fear","happy","neutral","sad","surprise"]
# Angry:958
# Disgust:111
# fear:1024
# happy:1774
# neutral:1233
# sad:1247
# surprise:831

def oversample_dataset(dataset):
    """
    Oversample each class so that each class has the same number of samples as the class with the maximum count.
    """
    labels = np.array(dataset.targets)
    unique_classes, class_counts = np.unique(labels, return_counts=True)
    #max_count = np.max(class_counts)
    max_count=750

    oversampled_indices = []
    for cls in unique_classes:
        # Get indices of all samples of this class.
        cls_indices = np.where(labels == cls)[0]
        # Randomly sample indices with replacement to match max_count.
        replicated_indices = np.random.choice(cls_indices, size=max_count, replace=True)
        oversampled_indices.extend(replicated_indices)
    # Shuffle the list of oversampled indices to mix classes
    random.shuffle(oversampled_indices)
    return Subset(dataset, oversampled_indices)

# Create an oversampled dataset and corresponding DataLoader:
oversampled_dataset = oversample_dataset(test_data)
print(len(oversampled_dataset))
test_dataloader=DataLoader(oversampled_dataset,batch_size=16,shuffle=True,num_workers=4,pin_memory=True,persistent_workers=True)


model=models.resnet50(pretrained=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

features=model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 7)
)

model.load_state_dict(torch.load("EmotionDetection.pth"))
model.to(device)
# Initialize lists to store predictions and labels
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    total=0
    correct=0
    for inputs,targets in test_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs=model(inputs)
        _,predicted=torch.max(outputs,1)
        preds,targets=predicted.cpu(),targets.cpu()
        all_preds.extend(preds.numpy())  # Store predictions
        all_labels.extend(targets.numpy())  # Store true labels

print(confusion_matrix(all_labels, all_preds))
print(classification_report(all_labels, all_preds, target_names=class_names))
print(f"Test Accuracy prediction:{accuracy_score(all_labels,all_preds)}")
print(f"Test Precision prediction:{precision_score(all_labels,all_preds, average='macro')}")
print(f"Test Recall prediction:{recall_score(all_labels,all_preds, average='macro')}")
print(f"Test f1-score prediction:{f1_score(all_labels,all_preds, average='macro')}")

5250


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[[455  19 100  14  78  75   9]
 [111 545  15  29  22  24   4]
 [ 79   9 331  11 108 128  84]
 [ 12   2   9 658  33  12  24]
 [ 51   4  34  38 527  80  16]
 [ 98   7  72  27 168 366  12]
 [ 21   0  42  25  16   8 638]]
              precision    recall  f1-score   support

       Angry       0.55      0.61      0.58       750
     Disgust       0.93      0.73      0.82       750
        fear       0.55      0.44      0.49       750
       happy       0.82      0.88      0.85       750
     neutral       0.55      0.70      0.62       750
         sad       0.53      0.49      0.51       750
    surprise       0.81      0.85      0.83       750

    accuracy                           0.67      5250
   macro avg       0.68      0.67      0.67      5250
weighted avg       0.68      0.67      0.67      5250

Test Accuracy prediction:0.6704761904761904
Test Precision prediction:0.6774242632179431
Test Recall prediction:0.6704761904761904
Test f1-score prediction:0.66955301950373


In [ ]:
import os
import cv2
import mediapipe as mp
import pandas as pd
from tqdm import tqdm  # For progress bar

# Paths
input_folder = "/content/drive/MyDrive/data/train_1/angry"  # Folder containing images
dir_path="/content/drive/MyDrive/extracted/angry/features.csv"
directory_csv = os.path.dirname(dir_path)

# Create the directory if it doesn't exist
if not os.path.exists(directory_csv):
    os.makedirs(directory_csv)

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)
mp_drawing = mp.solutions.drawing_utils

# List to store features
all_features = []

# Process each image
image_files = [f for f in os.listdir(input_folder)]

for image_file in tqdm(image_files, desc="Extracting Features"):
    image_path = os.path.join(input_folder, image_file)
    image = cv2.imread(image_path)

    # Process the image with MediaPipe
    result = face_mesh.process(image)

    # Extract landmarks if found
    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            # Extract the 468 landmarks (x, y, z coordinates)
            features = []
            for landmark in face_landmarks.landmark:
                features.extend([landmark.x, landmark.y, landmark.z])  # Add x, y, z coordinates

            # Append features with image filename
            all_features.append([image_file] + features)

# Convert to DataFrame and save to CSV
columns = ['image'] + [f'point_{i}_{axis}' for i in range(468) for axis in ['x', 'y', 'z']]
df = pd.DataFrame(all_features, columns=columns)
df.to_csv(dir_path, index=False)

print(f"Features saved to {directory_csv}")


ModuleNotFoundError: No module named 'mediapipe'

In [ ]:
import pandas as pd
angry_df=pd.read_csv(r"/content/drive/MyDrive/extracted/angry/features.csv")
angry_df["emotion"]="angry"
disgust_df=pd.read_csv(r"/content/drive/MyDrive/extracted/disgust/features.csv")
disgust_df["emotion"]="disgust"
fear_df=pd.read_csv(r"/content/drive/MyDrive/extracted/fear/features.csv")
fear_df["emotion"]="fear"
happy_df=pd.read_csv(r"/content/drive/MyDrive/extracted/happy/features.csv")
happy_df["emotion"]="happy"
neutral_df=pd.read_csv(r"/content/drive/MyDrive/extracted/neutral/features.csv")
neutral_df["emotion"]="neutral"
sad_df=pd.read_csv(r"/content/drive/MyDrive/extracted/sad/features.csv")
sad_df["emotion"]="sad"
surprise_df=pd.read_csv(r"/content/drive/MyDrive/extracted/surprise/features.csv")
surprise_df["emotion"]="surprise"
df=pd.concat([angry_df,disgust_df,fear_df,happy_df,neutral_df,sad_df,surprise_df],axis=0)
df.to_pickle("dataframe.pkl")

In [ ]:
from sklearn.ensemble import IsolationForest
df=pd.read_pickle("dataframe.pkl")
x=df.drop(["image","emotion"],axis=1)
y=df["emotion"]

# Isolation Forest for outlier detection
iso_forest = IsolationForest(n_estimators=100, contamination=0.02, random_state=42)
outlier_preds = iso_forest.fit_predict(x)

# Add predictions to dataframe
df['outlier'] = outlier_preds
df_clean = df[df['outlier'] == 1].drop('outlier', axis=1)

In [ ]:
import pandas as pd
import torch
import torchvision
from torch import optim,nn
from torch.utils.data import DataLoader,TensorDataset
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
import pickle
import numpy as np


x=df_clean.drop(["image","emotion"],axis=1)
y=df_clean["emotion"]

# smote = SMOTE()
# x_resampled, y_resampled = smote.fit_resample(x, y)

scaler=StandardScaler()
x_scaled=scaler.fit_transform(x)

with open('/content/drive/MyDrive/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

pca=PCA(n_components=256)
x_reduced=pca.fit_transform(x_scaled)
with open('/content/drive/MyDrive/pca.pkl', 'wb') as f:
    pickle.dump(pca, f)

encoder=LabelEncoder()
y_encoded=encoder.fit_transform(y)

x_tensor=torch.tensor(x_reduced,dtype=torch.float32)
y_tensor=torch.tensor(y_encoded,dtype=torch.long)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_counts = np.bincount(y_tensor.numpy())
class_weights = 1. / class_counts
weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)

class EmotionDetection(nn.Module):
    def __init__(self, input_dim):
        super(EmotionDetection, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 7)
        )

    def forward(self, x):
        return self.model(x)


model=EmotionDetection(input_dim=x_reduced.shape[1])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#criterion = nn.CrossEntropyLoss()

optimizer=optim.Adam(model.parameters(),lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

dataset = TensorDataset(x_tensor,y_tensor)
dataloader=DataLoader(dataset,batch_size=64,shuffle=True,num_workers=4,pin_memory=True)

epoch=51
model.to(device)

for i in range(epoch):
    model.train()
    running_loss=0
    all_preds=[]
    all_labels=[]
    for inputs,targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=criterion(outputs,targets)
        running_loss += loss.item() * inputs.size(0)
        _,preds=torch.max(outputs,1)
        preds,targets=preds.cpu(),targets.cpu()
        all_preds.extend(preds.numpy())  # Store predictions
        all_labels.extend(targets.numpy())  # Store true labels
        loss.backward()
        optimizer.step()
    epoch_loss = running_loss / len(dataloader.dataset)
    scheduler.step(epoch_loss)
    if i%10==0:
      print(f"{i}:Train Accuracy prediction:{accuracy_score(all_labels,all_preds)}")

print(f"{i}:Train Accuracy prediction:{accuracy_score(all_labels,all_preds)}")
print(f"Train Precision prediction:{precision_score(all_labels,all_preds, average='macro')}")
print(f"Train Recall prediction:{recall_score(all_labels,all_preds, average='macro')}")
print(f"Train f1-score prediction:{f1_score(all_labels,all_preds, average='macro')}")

torch.save(model.state_dict(), "EmotionDetection.pth")#to save the model and no need to train the model again and againprint(f"{i}/{epoch} loss-->{running_loss}")
print("Model saved")

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


0:Train Accuracy prediction:0.2910496557559906
10:Train Accuracy prediction:0.4940574637486057
20:Train Accuracy prediction:0.5225970229624216
30:Train Accuracy prediction:0.5457902227008731
40:Train Accuracy prediction:0.5616369860379246
50:Train Accuracy prediction:0.5716373706681026
50:Train Accuracy prediction:0.5716373706681026
Train Precision prediction:0.511164960583431
Train Recall prediction:0.5953236817145495
Train f1-score prediction:0.5276356098266282
Model saved


In [ ]:
#mediapipe testdata
import os
import cv2
import mediapipe as mp
import pandas as pd
from tqdm import tqdm  # For progress bar

# Paths
input_folder = "/content/drive/MyDrive/data/test_1/surprise"  # Folder containing images
dir_path="/content/drive/MyDrive/extracted_test/surprise/features.csv"
directory_csv = os.path.dirname(dir_path)

# Create the directory if it doesn't exist
if not os.path.exists(directory_csv):
    os.makedirs(directory_csv)

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)
mp_drawing = mp.solutions.drawing_utils

# List to store features
all_features = []

# Process each image
image_files = [f for f in os.listdir(input_folder)]

for image_file in tqdm(image_files, desc="Extracting Features"):
    image_path = os.path.join(input_folder, image_file)
    image = cv2.imread(image_path)

    # Process the image with MediaPipe
    result = face_mesh.process(image)

    # Extract landmarks if found
    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            # Extract the 468 landmarks (x, y, z coordinates)
            features = []
            for landmark in face_landmarks.landmark:
                features.extend([landmark.x, landmark.y, landmark.z])  # Add x, y, z coordinates

            # Append features with image filename
            all_features.append([image_file] + features)

# Convert to DataFrame and save to CSV
columns = ['image'] + [f'point_{i}_{axis}' for i in range(468) for axis in ['x', 'y', 'z']]
df = pd.DataFrame(all_features, columns=columns)
df.to_csv(dir_path, index=False)

print(f"Features saved to {directory_csv}")


Extracting Features: 100%|██████████| 841/841 [00:21<00:00, 39.01it/s]


Features saved to /content/drive/MyDrive/extracted_test/surprise


In [ ]:
import pandas as pd
angry_df=pd.read_csv(r"/content/drive/MyDrive/extracted_test/angry/features.csv")
angry_df["emotion"]="angry"
disgust_df=pd.read_csv(r"/content/drive/MyDrive/extracted_test/disgust/features.csv")
disgust_df["emotion"]="disgust"
fear_df=pd.read_csv(r"/content/drive/MyDrive/extracted_test/fear/features.csv")
fear_df["emotion"]="fear"
happy_df=pd.read_csv(r"/content/drive/MyDrive/extracted_test/happy/features.csv")
happy_df["emotion"]="happy"
neutral_df=pd.read_csv(r"/content/drive/MyDrive/extracted_test/neutral/features.csv")
neutral_df["emotion"]="neutral"
sad_df=pd.read_csv(r"/content/drive/MyDrive/extracted_test/sad/features.csv")
sad_df["emotion"]="sad"
surprise_df=pd.read_csv(r"/content/drive/MyDrive/extracted_test/surprise/features.csv")
surprise_df["emotion"]="surprise"
df=pd.concat([angry_df,disgust_df,fear_df,happy_df,neutral_df,sad_df,surprise_df],axis=0)
df.to_pickle("dataframe_test.pkl")

In [ ]:
import pandas as pd
import torch
import torchvision
from torch import optim,nn
from torch.utils.data import DataLoader,TensorDataset
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score
from imblearn.over_sampling import SMOTE

df=pd.read_pickle("dataframe_test.pkl")
x=df.drop(["image","emotion"],axis=1)
y=df["emotion"]


with open('/content/drive/MyDrive/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
x_scaled=scaler.transform(x)

with open('/content/drive/MyDrive/pca.pkl', 'rb') as f:
    pca = pickle.load(f)
x_reduced=pca.transform(x_scaled)

encoder=LabelEncoder()
y_encoded=encoder.fit_transform(y)

x_tensor=torch.tensor(x_reduced,dtype=torch.float32)
y_tensor=torch.tensor(y_encoded,dtype=torch.long)

class EmotionDetection(nn.Module):
    def __init__(self, input_dim):
        super(EmotionDetection, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 7)
        )

    def forward(self, x):
        return self.model(x)

model=EmotionDetection(input_dim=x_reduced.shape[1])
model.load_state_dict(torch.load("EmotionDetection.pth"))
optimizer=optim.Adam(model.parameters(),lr=1e-3, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = TensorDataset(x_tensor,y_tensor)
test_dataloader=DataLoader(dataset,batch_size=32,shuffle=True,num_workers=4,pin_memory=True)
model.to(device)
# Initialize lists to store predictions and labels
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    total=0
    correct=0
    for inputs,targets in test_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs=model(inputs)
        _,predicted=torch.max(outputs,1)
        preds,targets=predicted.cpu(),targets.cpu()
        all_preds.extend(preds.numpy())  # Store predictions
        all_labels.extend(targets.numpy())  # Store true labels

print(f"Test Accuracy prediction:{accuracy_score(all_labels,all_preds)}")
print(f"Test Precision prediction:{precision_score(all_labels,all_preds, average='macro')}")
print(f"Test Recall prediction:{recall_score(all_labels,all_preds, average='macro')}")
print(f"Test f1-score prediction:{f1_score(all_labels,all_preds, average='macro')}")

Test Accuracy prediction:0.5526949713941584
Test Precision prediction:0.4789707039154897
Test Recall prediction:0.5463435234178886
Test f1-score prediction:0.4903609266135435


In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 29.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.7 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is inco